# **MARKETING DATA ANALYSIS**
    

Problem:
The CEO of the Gamegit company wants to analyze the behavior of 4 campaigns launched for two different games.
For this, in the database I have 3 tables from which I can obtain the necessary data to put together a good analysis. After performing the ETL and creating the business tables, it is necessary to create a dashboard in Tableau with the most important metrics for the analysis.

In [1]:
# Import Libreries
import pandas as pd

In [2]:
# Import DataSets
campaign_ds = pd.read_csv('/Users/admin/Desktop/BKP/BetterBusiness/CAMPAIGN_SPEND.csv')
userdetail_ds = pd.read_csv('/Users/admin/Desktop/BKP/BetterBusiness/USER_DETAIL.csv')
userperformance_ds = pd.read_csv('/Users/admin/Desktop/BKP/BetterBusiness/USER_PERFORMANCE.csv')

## **Data Evaluation**

In [3]:
campaign_ds.head()

,APP,ACQUISITION_CHANNEL,PLATFORM,CAMPAIGN,CREATIVES_NAME,COUNTRY,SPEND_DATE,COST_IN_CENTS
0,jumping blocks,GOOGLE,ios,JP_03_campaign_API,jumping_blocks_ad1,au,2010-11-02,698.754545
1,jumping blocks,FACEBOOK,ios,JP_01_campaign_API,jumping_blocks_ad1,us,2010-11-02,56421.870835
2,jumping blocks,FACEBOOK,ios,JP_01_campaign_API,jumping_blocks_ad1,us,2010-11-03,110838.651390
3,parking hoops,FACEBOOK,android,PH_02_campaign_API,parking_hoops_ad6,gp,2010-11-02,10946.458559
4,parking hoops,FACEBOOK,android,PH_02_campaign_API,parking_hoops_ad6,jp,2010-11-03,21236.584923


In [4]:
campaign_ds.dtypes

APP                     object
ACQUISITION_CHANNEL     object
PLATFORM                object
CAMPAIGN                object
CREATIVES_NAME          object
COUNTRY                 object
SPEND_DATE              object
COST_IN_CENTS          float64
dtype: object

In [5]:
userdetail_ds.head()

,PSEUDO_USER_ID,APP_NAME,PLATFORM,ACQUISITION_CHANNEL,CAMPAIGN_NAME,CREATIVES_NAME,COUNTRY,VERSION,INSTALLATION_DATE
0,3ecc3c85e927513a0efb861d1ccc96fc,jumping blocks,ios,Google,jp_03_campaign,jumping_blocks_ad2,jp,0.48.1,2010-11-02
1,244edbb9f7579b61a10a7cbf1caa8fd9,jumping blocks,ios,Facebook,jp_01_campaign,jumping_blocks_ad1,us,0.40.1,2010-11-02
2,5636cd6d764866ee740180176c7fc112,jumping blocks,ios,Facebook,jp_01_campaign,jumping_blocks_ad1,us,0.48.1,2010-11-02
3,0cbb351d42ec80509283cfe62aa15545,jumping blocks,ios,Google,jp_03_campaign,jumping_blocks_ad1,au,0.40.1,2010-11-01
4,eb3e1391fcdce217d7c5b777b425702c,jumping blocks,ios,Facebook,jp_01_campaign,jumping_blocks_ad1,us,0.48.1,2010-11-02


In [6]:
userdetail_ds.dtypes

PSEUDO_USER_ID         object
APP_NAME               object
PLATFORM               object
ACQUISITION_CHANNEL    object
CAMPAIGN_NAME          object
CREATIVES_NAME         object
COUNTRY                object
VERSION                object
INSTALLATION_DATE      object
dtype: object

In [7]:
userperformance_ds.head()

,ID,APP_NAME,PLATFORM,DATE,REVENUE_CENTS,IMPRESSIONS
0,0cbb351d42ec80509283cfe62aa15545,jumping blocks,ios,2010-11-02,6.265643,120
1,7ba7e9e0b10e375f5443636dc6211403,jumping blocks,ios,2010-11-02,10.434608,230
2,771755d87a237bb36f38c6ee14b99c0b,jumping blocks,ios,2010-11-02,17.246593,390
3,2ba2aacf4bce2ea516be1c4b9ae0241c,jumping blocks,ios,2010-11-03,77.863457,1760
4,2e095e9ab510c5b51d5403fa5f024283,jumping blocks,ios,2010-11-02,126.977463,2870


In [8]:
userperformance_ds.dtypes

ID                object
APP_NAME          object
PLATFORM          object
DATE              object
REVENUE_CENTS    float64
IMPRESSIONS        int64
dtype: object

DataBase Tables:

- USER_DETAIL: It has information about the campaign by which each player came to the game, opened by game and with other attributes such as country, channel, creative, among others.

- CAMPAIGN_SPEND: Contains information about campaigns and their costs

- USER_PERFORMANCE: It contains information about the behavior of each player and the revenue they are generating with impressions.

## **Data Cleansing & Transformation**

First I am going to normalize the name of the columns of the different datasets
- PSEUDO_USER_ID from userdetail_ds and ID from userperformance_ds to USER_ID
- CAMPAIGN from campaign_ds to CAMPAIGN_NAME
- APP from campaign_ds to APP_NAME

In [51]:
campaign_ds = campaign_ds.rename(columns={'CAMPAIGN':'CAMPAIGN_NAME'})
campaign_ds = campaign_ds.rename(columns={'APP':'APP_NAME'})
userdetail_ds = userdetail_ds.rename(columns={'PSEUDO_USER_ID':'USER_ID'})
userperformance_ds = userperformance_ds.rename(columns={'ID':'USER_ID'})

In [10]:
campaign_ds_cv = campaign_ds['CAMPAIGN_NAME'].unique().tolist()
userdetail_ds_cv = userdetail_ds['CAMPAIGN_NAME'].unique().tolist()
print(campaign_ds_cv)
print(userdetail_ds_cv)

['JP_03_campaign_API', 'JP_01_campaign_API', 'PH_02_campaign_API', 'PH_01_campaign_API']
['jp_03_campaign', 'jp_01_campaign', 'ph_01_campaign', 'ph_02_campaign']


We can see that the campaign_name are different in the data sets. To make it easier to do the join later, we are going to transform the campaign_name of the campaign data set by removing the "_API" from the name and convert it to lowercase leaving it equal to the campaign name of user_detail

In [11]:
campaign_ds['CAMPAIGN_NAME'] = campaign_ds['CAMPAIGN_NAME'].str.replace('_API','')
campaign_ds['CAMPAIGN_NAME'] = campaign_ds['CAMPAIGN_NAME'].str.lower()
campaign_ds_cv = campaign_ds['CAMPAIGN_NAME'].unique().tolist()
print(campaign_ds_cv)

['jp_03_campaign', 'jp_01_campaign', 'ph_02_campaign', 'ph_01_campaign']


In [34]:
# DROP DUPLICATES IN USER_DETAIL

#check duplicates
key = ["USER_ID",'APP_NAME','PLATFORM','ACQUISITION_CHANNEL','CAMPAIGN_NAME','CREATIVES_NAME','COUNTRY','VERSION','INSTALLATION_DATE']
userdetail_ds.fillna(-999).groupby(key)['USER_ID'].count().sort_values(ascending=False).head(20)

# drop duplicates based on an subset of variables.
key = ["USER_ID"]
df_dedupped2 = userdetail_ds.drop_duplicates(subset=key)

## **Data Merge**

### LTV DataSet

In [35]:
userperformance_ds.head()

,USER_ID,APP_NAME,PLATFORM,DATE,REVENUE_CENTS,IMPRESSIONS
0,0cbb351d42ec80509283cfe62aa15545,jumping blocks,ios,2010-11-02,6.265643,120
1,7ba7e9e0b10e375f5443636dc6211403,jumping blocks,ios,2010-11-02,10.434608,230
2,771755d87a237bb36f38c6ee14b99c0b,jumping blocks,ios,2010-11-02,17.246593,390
3,2ba2aacf4bce2ea516be1c4b9ae0241c,jumping blocks,ios,2010-11-03,77.863457,1760
4,2e095e9ab510c5b51d5403fa5f024283,jumping blocks,ios,2010-11-02,126.977463,2870


In [36]:
userdetail_ds.head()

,USER_ID,APP_NAME,PLATFORM,ACQUISITION_CHANNEL,CAMPAIGN_NAME,CREATIVES_NAME,COUNTRY,VERSION,INSTALLATION_DATE
0,3ecc3c85e927513a0efb861d1ccc96fc,jumping blocks,ios,Google,jp_03_campaign,jumping_blocks_ad2,jp,0.48.1,2010-11-02
1,244edbb9f7579b61a10a7cbf1caa8fd9,jumping blocks,ios,Facebook,jp_01_campaign,jumping_blocks_ad1,us,0.40.1,2010-11-02
2,5636cd6d764866ee740180176c7fc112,jumping blocks,ios,Facebook,jp_01_campaign,jumping_blocks_ad1,us,0.48.1,2010-11-02
3,0cbb351d42ec80509283cfe62aa15545,jumping blocks,ios,Google,jp_03_campaign,jumping_blocks_ad1,au,0.40.1,2010-11-01
4,eb3e1391fcdce217d7c5b777b425702c,jumping blocks,ios,Facebook,jp_01_campaign,jumping_blocks_ad1,us,0.48.1,2010-11-02


In [37]:
# MERGE OF USER_DETAIL AND USER_PERFORMANCE
life_time_value_ds = pd.merge(userdetail_ds, userperformance_ds)

In [38]:
life_time_value_ds.head()

,USER_ID,APP_NAME,PLATFORM,ACQUISITION_CHANNEL,CAMPAIGN_NAME,CREATIVES_NAME,COUNTRY,VERSION,INSTALLATION_DATE,DATE,REVENUE_CENTS,IMPRESSIONS
0,3ecc3c85e927513a0efb861d1ccc96fc,jumping blocks,ios,Google,jp_03_campaign,jumping_blocks_ad2,jp,0.48.1,2010-11-02,2010-11-03,56.793863,170
1,3ecc3c85e927513a0efb861d1ccc96fc,jumping blocks,ios,Google,jp_03_campaign,jumping_blocks_ad2,jp,0.48.1,2010-11-02,2010-11-02,69.414702,230
2,244edbb9f7579b61a10a7cbf1caa8fd9,jumping blocks,ios,Facebook,jp_01_campaign,jumping_blocks_ad1,us,0.40.1,2010-11-02,2010-11-02,11.881267,270
3,5636cd6d764866ee740180176c7fc112,jumping blocks,ios,Facebook,jp_01_campaign,jumping_blocks_ad1,us,0.48.1,2010-11-02,2010-11-02,42.387667,970
4,0cbb351d42ec80509283cfe62aa15545,jumping blocks,ios,Google,jp_03_campaign,jumping_blocks_ad1,au,0.40.1,2010-11-01,2010-11-02,6.265643,120


In [39]:
# RENAME THE DATE COLUMN TO REVENUE_DATE
life_time_value_ds = life_time_value_ds.rename(columns={'DATE':'REVENUE_DATE'})

In [40]:
life_time_value_ds['INSTALLS'] = 1
life_time_value_ds.head()

,USER_ID,APP_NAME,PLATFORM,ACQUISITION_CHANNEL,CAMPAIGN_NAME,CREATIVES_NAME,COUNTRY,VERSION,INSTALLATION_DATE,REVENUE_DATE,REVENUE_CENTS,IMPRESSIONS,INSTALLS
0,3ecc3c85e927513a0efb861d1ccc96fc,jumping blocks,ios,Google,jp_03_campaign,jumping_blocks_ad2,jp,0.48.1,2010-11-02,2010-11-03,56.793863,170,1
1,3ecc3c85e927513a0efb861d1ccc96fc,jumping blocks,ios,Google,jp_03_campaign,jumping_blocks_ad2,jp,0.48.1,2010-11-02,2010-11-02,69.414702,230,1
2,244edbb9f7579b61a10a7cbf1caa8fd9,jumping blocks,ios,Facebook,jp_01_campaign,jumping_blocks_ad1,us,0.40.1,2010-11-02,2010-11-02,11.881267,270,1
3,5636cd6d764866ee740180176c7fc112,jumping blocks,ios,Facebook,jp_01_campaign,jumping_blocks_ad1,us,0.48.1,2010-11-02,2010-11-02,42.387667,970,1
4,0cbb351d42ec80509283cfe62aa15545,jumping blocks,ios,Google,jp_03_campaign,jumping_blocks_ad1,au,0.40.1,2010-11-01,2010-11-02,6.265643,120,1


In [41]:
# AGREGATIONS
life_time_value_ds = life_time_value_ds.groupby(['APP_NAME','PLATFORM','ACQUISITION_CHANNEL',
'CAMPAIGN_NAME','CREATIVES_NAME','COUNTRY','INSTALLATION_DATE','REVENUE_DATE']).sum()

In [44]:
life_time_value_ds.to_csv('/Users/admin/Desktop/BKP/BetterBusiness/LTV.csv')

### Spend & Revenue DataSet

In [45]:
userperformance_ds.head()

,USER_ID,APP_NAME,PLATFORM,DATE,REVENUE_CENTS,IMPRESSIONS
0,0cbb351d42ec80509283cfe62aa15545,jumping blocks,ios,2010-11-02,6.265643,120
1,7ba7e9e0b10e375f5443636dc6211403,jumping blocks,ios,2010-11-02,10.434608,230
2,771755d87a237bb36f38c6ee14b99c0b,jumping blocks,ios,2010-11-02,17.246593,390
3,2ba2aacf4bce2ea516be1c4b9ae0241c,jumping blocks,ios,2010-11-03,77.863457,1760
4,2e095e9ab510c5b51d5403fa5f024283,jumping blocks,ios,2010-11-02,126.977463,2870


In [47]:
revenueds = pd.merge(userdetail_ds, userperformance_ds)
revenueds.head()

,USER_ID,APP_NAME,PLATFORM,ACQUISITION_CHANNEL,CAMPAIGN_NAME,CREATIVES_NAME,COUNTRY,VERSION,INSTALLATION_DATE,DATE,REVENUE_CENTS,IMPRESSIONS
0,3ecc3c85e927513a0efb861d1ccc96fc,jumping blocks,ios,Google,jp_03_campaign,jumping_blocks_ad2,jp,0.48.1,2010-11-02,2010-11-03,56.793863,170
1,3ecc3c85e927513a0efb861d1ccc96fc,jumping blocks,ios,Google,jp_03_campaign,jumping_blocks_ad2,jp,0.48.1,2010-11-02,2010-11-02,69.414702,230
2,244edbb9f7579b61a10a7cbf1caa8fd9,jumping blocks,ios,Facebook,jp_01_campaign,jumping_blocks_ad1,us,0.40.1,2010-11-02,2010-11-02,11.881267,270
3,5636cd6d764866ee740180176c7fc112,jumping blocks,ios,Facebook,jp_01_campaign,jumping_blocks_ad1,us,0.48.1,2010-11-02,2010-11-02,42.387667,970
4,0cbb351d42ec80509283cfe62aa15545,jumping blocks,ios,Google,jp_03_campaign,jumping_blocks_ad1,au,0.40.1,2010-11-01,2010-11-02,6.265643,120


In [57]:
revenueds['INSTALLS'] = 1
revenue_ds = revenueds.groupby(['APP_NAME','PLATFORM','ACQUISITION_CHANNEL','CAMPAIGN_NAME','CREATIVES_NAME','COUNTRY','INSTALLATION_DATE']).sum()

In [80]:
campaign_ds.head()

,APP_NAME,ACQUISITION_CHANNEL,PLATFORM,CAMPAIGN_NAME,CREATIVES_NAME,COUNTRY,SPEND_DATE,COST_IN_CENTS
0,jumping blocks,GOOGLE,ios,jp_03_campaign,jumping_blocks_ad1,au,2010-11-02,698.754545
1,jumping blocks,FACEBOOK,ios,jp_01_campaign,jumping_blocks_ad1,us,2010-11-02,56421.870835
2,jumping blocks,FACEBOOK,ios,jp_01_campaign,jumping_blocks_ad1,us,2010-11-03,110838.651390
3,parking hoops,FACEBOOK,android,ph_02_campaign,parking_hoops_ad6,gp,2010-11-02,10946.458559
4,parking hoops,FACEBOOK,android,ph_02_campaign,parking_hoops_ad6,jp,2010-11-03,21236.584923


In [81]:
cost_ds = campaign_ds

In [82]:
cost_ds = cost_ds.rename(columns={'SPEND_DATE':'INSTALLATION_DATE'})

In [83]:
cost_ds = cost_ds.groupby(['APP_NAME','PLATFORM','ACQUISITION_CHANNEL','CAMPAIGN_NAME','CREATIVES_NAME','COUNTRY','INSTALLATION_DATE']).sum()

In [90]:
cost_ds.head()

COST_IN_CENTS
APP_NAME       PLATFORM ACQUISITION_CHANNEL CAMPAIGN_NAME  CREATIVES_NAME     COUNTRY INSTALLATION_DATE               
jumping blocks ios      FACEBOOK            jp_01_campaign jumping_blocks_ad1 au      2010-11-01            113.970348
                                                                                      2010-11-02             12.929688
                                                                                      2010-11-03            180.313427
                                                                              ca      2010-11-01            228.944149
                                                                                      2010-11-02             29.736776

In [96]:
spendrevenue_ds = revenue_ds.merge(cost_ds, how='outer',left_on=['APP_NAME','PLATFORM','ACQUISITION_CHANNEL','CAMPAIGN_NAME','CREATIVES_NAME','COUNTRY','INSTALLATION_DATE'],right_on=['APP_NAME','PLATFORM','ACQUISITION_CHANNEL','CAMPAIGN_NAME','CREATIVES_NAME','COUNTRY','INSTALLATION_DATE'])

In [97]:
spendrevenue_ds.head()

REVENUE_CENTS  \
APP_NAME       PLATFORM ACQUISITION_CHANNEL CAMPAIGN_NAME  CREATIVES_NAME     COUNTRY INSTALLATION_DATE                  
jumping blocks ios      Facebook            jp_01_campaign jumping_blocks_ad1 au      2010-11-01            140.704133   
                                                                                      2010-11-02             28.696097   
                                                                                      2010-11-03            118.706745   
                                                                              ca      2010-11-01            384.176075   
                                                                                      2010-11-02             76.177429   

                                                                                                         IMPRESSIONS  \
APP_NAME       PLATFORM ACQUISITION_CHANNEL CAMPAIGN_NAME  CREATIVES_NAME     COUNTRY INSTALLATION_DATE                
jumping blocks ios      Facebook            jp_01_campaign jumping_blocks_ad1 au      2010-11-01              3200.0   
                                                                                      2010-11-02               630.0   
                                                                                      2010-11-03              2690.0   
                                                                              ca      2010-11-01              8750.0   
                                                                                      2010-11-02              1730.0   

                                                                                                         INSTALLS  \
APP_NAME       PLATFORM ACQUISITION_CHANNEL CAMPAIGN_NAME  CREATIVES_NAME     COUNTRY INSTALLATION_DATE             
jumping blocks ios      Facebook            jp_01_campaign jumping_blocks_ad1 au      2010-11-01              5.0   
                                                                                      2010-11-02              2.0   
                                                                                      2010-11-03              3.0   
                                                                              ca      2010-11-01             14.0   
                                                                                      2010-11-02              4.0   

                                                                                                         COST_IN_CENTS  
APP_NAME       PLATFORM ACQUISITION_CHANNEL CAMPAIGN_NAME  CREATIVES_NAME     COUNTRY INSTALLATION_DATE                 
jumping blocks ios      Facebook            jp_01_campaign jumping_blocks_ad1 au      2010-11-01                   NaN  
                                                                                      2010-11-02                   NaN  
                                                                                      2010-11-03                   NaN  
                                                                              ca      2010-11-01                   NaN  
                                                                                      2010-11-02                   NaN

In [98]:
spendrevenue_ds.to_csv('/Users/admin/Desktop/BKP/BetterBusiness/Spend & Revenue.csv')